# Kubernetes Management Tasks

This is a demo notebook for common kubernetes tasks.

*Note: You should be aware of where and how the jupyter session is running, as kubernetes credentials are sensitive information and leakage posts security risks.*

## Configure kubectl

If you did not start this notebook with pre-configured KUBECONFIG, you may paste your `kubectl config view --raw --minify` output here.  Otherwise skip to the next section.

Notes:
1. if you use gke, your `kubectl config` contains a token that a valid for an hour. Run `kubectl get cs` to refresh token locally before copying the config.

In [ ]:
import tempfile
import ipywidgets
import os

In [ ]:
kubeconfig = ipywidgets.Textarea(description='kubeconfig:', placeholder='kubectl config view --raw --minify | pbcopy')
kubeconfig

In [ ]:
temp = tempfile.NamedTemporaryFile(delete=False)
temp.write(bytearray(kubeconfig.value, 'utf-8'))
temp.close()

os.environ['KUBECONFIG'] = temp.name

## Check version and cluster health

In [ ]:
!! ~/bin/kubectl version

In [ ]:
!! ~/bin/kubectl get cs

In [ ]:
!! ~/bin/kubectl get node

## Delete user pvc

In [ ]:
user_pvcs = !! ~/bin/kubectl get -l component=singleuser-storage  pvc -n hub -o jsonpath='{range.items[*]}{.metadata.name}{"\n"}{end}'

pvc = ipywidgets.Dropdown(
    options=user_pvcs,
    description='Target:',
    disabled=False,
)
pvc

In [ ]:
mounted_by = !! ~/bin/kubectl -n hub describe pvc {pvc.value} | grep -o 'jupyter-.*'
mounted_by

if mounted_by:
    print("PVC %s can't be deleted because it's mounted by: %s" % (pvc.value, mounted_by[0]))
else:
    result = !! ~/bin/kubectl -n hub delete pvc {pvc.value}
    print(result)

## Create pv-type dataset

In [ ]:
pv_type_datasets = !! ~/bin/kubectl -n hub get datasets -o=custom-columns=VOLUME_NAME:spec.volumeName,NAME:metadata.name,TYPE:spec.type | grep -e 'pv$' | grep -v -e '^hostpath:' | awk '{print $1","$2}'
temp_datasets = {}
for ds in pv_type_datasets:
    [volume_name, name] = ds.split(',')
    temp_datasets[name] = volume_name
dataset_name = ipywidgets.Dropdown(
    options=temp_datasets.keys(),
    description='Target:',
    disabled=False,
)
dataset_name

In [ ]:
dataset_size = ipywidgets.IntText(description='Size:', value=200)
dataset_size

In [ ]:
is_rook_block = !! ~/bin/kubectl get sc | grep rook-block

storage_class = 'rook-block' if is_rook_block else 'standard'
storage_class

In [ ]:
dataset_volume = temp_datasets[dataset_name.value]

pv_type_dataset_yaml_string = '''apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  annotations:
    primehub-group: dataset-%s
    primehub-group-sc: %s
  name: dataset-%s
  namespace: hub
spec:
  accessModes:
  - ReadWriteMany
  dataSource: null
  resources:
    requests:
      storage: %s
  selector:
    matchLabels:
      primehub-group: dataset-%s
      primehub-namespace: hub
  storageClassName: ''
''' % (dataset_volume, storage_class, dataset_volume, str(dataset_size.value)+'Gi', dataset_volume)
print(pv_type_dataset_yaml_string)

In [ ]:
!! echo "{pv_type_dataset_yaml_string}" | ~/bin/kubectl -n hub apply -f -